In [3]:
import numpy as np

from sklearn.datasets import load_diabetes

# Neural network modules
from pynet.nn.sequential import Sequential
from pynet.nn.linear import Linear
from pynet.nn.relu import ReLU

# Datasets
from pynet.data.in_memory import InMemoryDataset

# Loss functions
from pynet.loss.mse import MeanSquaredError

# Optimizers
from pynet.optimizers.sgd import SGD

# Weight initializers
from pynet.initializers.he_normal import HeNormal

# Trainer and training/testing callbacks
from pynet.training.trainer.default import DefaultTrainer
from pynet.training.callbacks.print import PrintCallback
from pynet.training.callbacks.lr_schedule import LrSchedule

In [4]:
def lr_schedule(epoch: int, lr: float) -> float:
    if epoch > 0 and epoch % 50 == 0:
        return lr * 0.1
    else:
        return lr

In [5]:
X, y = load_diabetes(return_X_y=True)

# Feature normalization
X_norm = (X - X.min(0)) / X.ptp(0)

# inputs to neural net must be of shape [n, 1]
X_norm = np.expand_dims(X_norm, axis=2)

In [6]:
epochs = 200

model = Sequential([
    Linear(inputs=10, neurons=32, initializer=HeNormal()),
    ReLU(),
    Linear(inputs=32, neurons=16, initializer=HeNormal()),
    ReLU(),
    Linear(inputs=16, neurons=1, initializer=HeNormal())
])

dataset = InMemoryDataset(X_norm, y)
loss_f = MeanSquaredError()
sgd = SGD(learning_rate=0.0001, momentum=0.9)
callbacks = [PrintCallback(), LrSchedule(optimizer=sgd, schedule=lr_schedule)]
trainer = DefaultTrainer()

In [ ]:
history = trainer.train(
    model=model,
    train_dataset=dataset,
    val_dataset=None,
    loss_f=loss_f,
    optimizer=sgd,
    epochs=epochs,
    callbacks=callbacks
)